In [240]:
import pandas as pd
import os
from dotenv import dotenv_values
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from datetime import date

In [241]:
ct = date.today()

df = pd.read_csv('Loinc.csv')
df = df[['Id', 'ParentId', 'Level', 'Code', 'Sequence', 'CodeText', 'Component']]
df['ParentId'] = df['ParentId'].astype('Int64')

sorted_df = df.sort_values(['Level', 'Sequence'])

In [243]:
def path(parentId, code) -> str:
    if pd.isna(parentId):
       return "\\DocumentOntology\\" + code
    row = (sorted_df[sorted_df["Id"] == parentId]).iloc[0]
    return path(row["ParentId"], str(row["Id"]) + '\\' + code)
    
    

In [244]:
list = []
visual = []
for index, row in sorted_df.iterrows():
    list.append(path(row["ParentId"], str(row["Id"])) + '\\')
    visual.append('FA' if 'LP' in row['Code'] else 'LA')
sorted_df['fullName'] = list     
sorted_df['visual'] = visual

In [248]:


dictionary = {
    "C_HLEVEL": map(lambda x: x + 2, sorted_df['Level'].tolist()),
    "C_FULLNAME": sorted_df['fullName'].tolist(),
    "C_NAME": sorted_df['CodeText'].tolist(),
    "C_SYNONYM_CD": 'N',
    "C_VISUALATTRIBUTES": visual,
    "C_TOTALNUM": pd.NA,
    "C_BASECODE": map(lambda x: 'LOINC:' + x, sorted_df['Code'].tolist()),
    "C_METADATAXML": pd.NA,
    "C_FACTTABLECOLUMN": 'concept_cd',
    "C_TABLENAME": 'concept_dimension',
    "C_COLUMNNAME": 'concept_path',
    "C_COLUMNDATATYPE": 'T',
    "C_OPERATOR": 'LIKE',
    "C_DIMCODE": sorted_df['fullName'].tolist(),
    "C_COMMENT": pd.NA,
    "C_TOOLTIP": pd.NA,
    "M_APPLIED_PATH": '@',
    "UPDATE_DATE": ct,
    "DOWNLOAD_DATE": ct,
    "IMPORT_DATE": ct,
    "SOURCESYSTEM_CD": 'MU',
    "VALUETYPE_CD": pd.NA,
    "M_EXCLUSION_CD": pd.NA,
    "C_PATH": pd.NA,
    "C_SYMBOL": pd.NA
    
}
final_df = pd.DataFrame(dictionary)


In [246]:

"""
db credential
"""
# read db credential
config = dotenv_values("./environment/.env") 

# make connection
print('Connecting to DB...')
connection = snowflake.connector.connect(
            user=config['USERNAME'],
            password=config['PASSWORD'],
            account=config['ACCOUNT'],
            warehouse=config['WAREHOUSE'],
            role=config['ROLE'],
            database=config['DATABASE'],
            schema=config['SCHEMA']
)
print('Connected...')


Connecting to DB...
Connected...


In [247]:
ont_table = 'DOCUMENT_ONTOLOGY'
print('Creating Table...')
cur = connection.cursor()
try:
    sql = f"""
     CREATE OR REPLACE TABLE {ont_table} (
        C_HLEVEL                              INT    NOT NULL,
        C_FULLNAME                    VARCHAR(700)   NOT NULL,
        C_NAME                                VARCHAR(2000)  NOT NULL,
        C_SYNONYM_CD                  CHAR(1) NOT NULL,
        C_VISUALATTRIBUTES    CHAR(3) NOT NULL,
        C_TOTALNUM                    INT    NULL,
        C_BASECODE                    VARCHAR(50)    NULL,
        C_METADATAXML                 TEXT    NULL,
        C_FACTTABLECOLUMN             VARCHAR(50)    NOT NULL,
        C_TABLENAME                   VARCHAR(50)    NOT NULL,
        C_COLUMNNAME                  VARCHAR(50)    NOT NULL,
        C_COLUMNDATATYPE              VARCHAR(50)    NOT NULL,
        C_OPERATOR                    VARCHAR(10)    NOT NULL,
        C_DIMCODE                             VARCHAR(700)   NOT NULL,
        C_COMMENT                             TEXT    NULL,
        C_TOOLTIP                             VARCHAR(900)   NULL,
        M_APPLIED_PATH                VARCHAR(700)   NOT NULL,
        UPDATE_DATE                   DATE    NOT NULL,
        DOWNLOAD_DATE                 DATE    NULL,
        IMPORT_DATE                   DATE    NULL,
        SOURCESYSTEM_CD               VARCHAR(50)    NULL,
        VALUETYPE_CD                  VARCHAR(50)    NULL,
        M_EXCLUSION_CD                        VARCHAR(25) NULL,
        C_PATH                                VARCHAR(700)   NULL,
        C_SYMBOL                              VARCHAR(50)    NULL
    ) 
    """
    cur.execute(sql)
       
finally:
    cur.close()
print('Loading Ontology...')
write_pandas(connection, final_df, ont_table)    
# Close connection
connection.close()
print('Done...')

Creating Table...
Loading Ontology...
Done...
